In [9]:
## imports

import numpy as np
import itertools
from collections import namedtuple
import os
import random
from random import shuffle
from collections import namedtuple

Combo = namedtuple('Combo', 'x solution')

In [10]:
## all functions
def read_data(filename):
    array = np.fromfile(filename, dtype=int, sep=' ')
    dim = array[0]
    obj = []
    for i in range(4):
        obj.append(array[1+i*dim**2:1+(i+1)*dim**2].reshape((dim, dim)))
    return dim, obj


def init(dim, n=1):
    init = list()
    for i in range(n):
        tmp = [i for i in range(dim)]
        shuffle(tmp)
        init.append(tmp)
    return init

def get_solution(x, objs):
    """ je considère qu'on ne me donne que un vecteur de solution , et pas un vecteur de vecteur de solution."""
    obj_to_return = []
    for obj in objs:
        calcul = 0
        for i in range(len(x)):
            calcul += obj[i][x[i]]
        obj_to_return.append(calcul)
    return obj_to_return


def findNeighbours(X, nbNeighbour):
  neighbours = []
  for i in range(0, nbNeighbour):
    newNeighbour = X.copy()
    while True:
      row1 = int(round(random.random()*len(X))) % len(X)
      row2 = int(round(random.random()*len(X))) % len(X)
      tmp = newNeighbour[row1]
      newNeighbour[row1] = newNeighbour[row2]
      newNeighbour[row2] = tmp
      
      found = False
      for nb in neighbours:
        doublon = True
        for j in range(0, len(X)):
          if nb[j] != newNeighbour[j]:
            doublon = False
        if doublon == True:
          found = True

      if not found:
        neighbours.append(newNeighbour)
        break
  return neighbours

def findNeighboursA(X, nbNeighbour,nPermut):
    neighbours = []
    for i in range(0, nbNeighbour):
        newNeighbour = X.copy()
    
        while True:
            
            permut_rows = random.sample(X,nPermut)
                                        
            temps = [newNeighbour[row] for row in permut_rows]
            
            for tmp,row in zip(reversed(temps),permut_rows) :   
                newNeighbour[row] = tmp

            found = False
            for nb in neighbours:
                doublon = True
                for j in range(0, len(X)):
                    if nb[j] != newNeighbour[j]:
                        doublon = False
                if doublon == True:
                    found = True

            if not found:
                neighbours.append(newNeighbour)
                break

    return neighbours


def x_and_sol_to_named_tuple(x_to_convert,objs) :
    
    """ Prend en entrée une liste de vecteurs X et 
    les associe à leurs solutions dans un named tuple 
    renvoit une liste de named_tuples """ 
    
    if (type(x_to_convert[0]) == list) :
        
        tuple_list = []
        for x in x_to_convert:
            #print(x)
            x_tuple = Combo(x,get_solution(x,objs))
            tuple_list.append(x_tuple)

        return tuple_list
    
    else :
        
        return [Combo(x_to_convert,get_solution(x_to_convert,objs))]
## TODO CHANGED HERE    

    
def compare_two_points(tuple1,tuple2,) :
    
    """ return an integer 0, 1 , 2 to determine order of dominance
    0 : no dominance
    1 : the first element gets dominated
    2 : the second element gets dominated """
    
    #print("first tuple", tuple1)
    #print("second tupel",tuple2)
    elem_to_del = 0
    
    solutions1 = tuple1.solution 
    solutions2 = tuple2.solution 
    
    comp_list = [-1 if obj1>obj2 else 0 if obj1==obj2 else 1 for obj1,obj2 in zip(solutions1,solutions2)]
    
    comp_set = set(comp_list)
    #print(comp_set)
    
    if all(x in [0,1] for x in comp_set) | ({1} == comp_set) :
        #print('on delete le 2e element : ' , tuple2)
        elem_to_del = 2
    
    elif (all(x in [0,-1] for x in comp_set)) | ({-1} == comp_set) :
        #print('on delete le 1er element :', tuple1)
        elem_to_del = 1
         
    elif {0} == comp_set :
        #print('on ne supprime rien, les obj sont egaux :')
        elem_to_del = 0
        
    else :
        #print('On ne supprime rien ')
        elem_to_del = 0
    
    return elem_to_del

def compare_and_delete(tuple_group) :
    
    """Compares each tuple to all the elements in the list,
    deletes the dominated ones,
    as well as the duplicates """

    dominated = []
    unique_domi = []
    unique_tuple_group = []
    tuple_group_copy = tuple_group.copy()
    
    ## creates a list of dominated tuples
    for (tuple1,tuple2) in itertools.combinations(tuple_group_copy,2):
             
            
            who_to_del = compare_two_points(tuple1,tuple2,)
         
            if who_to_del == 1 :
                dominated.append(tuple1)
            if who_to_del == 2 :
                dominated.append(tuple2)
    
    ## creates unique list of dominated tuples
    unique = [unique_domi.append(x) for x in dominated if x not in unique_domi]         
    #print('the dominated vector' , unique_domi)
    
    ## delete all the dominated tuples   
    tuple_group_copy[:] =  [n for n in tuple_group_copy if n not in unique_domi]
    
    ### delete all the duplicates in the tuple list  
    unique2 = [unique_tuple_group.append(x) for x in tuple_group_copy if x not in unique_tuple_group]
         
    return unique_tuple_group
    


def add_and_update_archive(neighbour_tuples, archive_tuple) :
    
    """ pour chaque nouvel élément potentiel, comparer cet élément à tout l'archive 
    Si l'élément domine un élément de l'archive, supprimer ce dernier et ajouter le nouvel élément
    Si l'élément se fait dominer au moins une fois, supprimer l'élement 
    Sinon, ajouter à l'archive."""
    
    who_to_del = 0 
    neigh_is_worse = False
    neighs_to_add = []
    unique_archive = []
   
    archive_tuple_copy = archive_tuple.copy()
    
    for neighbour in neighbour_tuples :
        
        
        for arch in reversed(archive_tuple_copy) : 
            
            #print('treating another arch')
                   
            who_to_del = compare_two_points(neighbour,arch)
            
            if who_to_del == 2 :
                
                #print("on va del qqun dans l'arch : ", arch)
                #print("celui qui l'a éliminé ", neighbour)
                archive_tuple_copy[:] = [x for x in archive_tuple_copy if x != arch]
            
            if who_to_del == 1 :
                #print(' issue : neighbour is worse than someone in the archive')
                neigh_is_worse = True
                break
                
        if neigh_is_worse == False :
            neighs_to_add.append(neighbour)
            
            neigh_is_worse = False
        
                
            #print('after boucle if \n')   
            #print(archive_tuple_copy)
    
    #print('neight to add : \n' ,neighs_to_add)
    archive_tuple_copy.extend(neighs_to_add)
    #print('full archive \n: ' , archive_tuple_copy)
    
    unique = [unique_archive.append(x) for x in archive_tuple_copy if x not in unique_archive]
    
    #print('type unique_arch ' , type(unique_archive))
    
    return unique_archive

In [11]:
def compare_old_and_new_archive(archive_old,archive_new) :
    
    equivalent_found = 0
    archive_changed = True;

    for new in archive_new :   
    
        for old in archive_old :    

            #print(old[0].x)
            #print(type(old))
            #print(new.x)

            if old.x == new.x :

                equivalent_found += 1
                continue;
    #print("number of equivalences found : ", equivalent_found)
    if (equivalent_found == len(archive_new) & len(archive_old) == len(archive_new)):
        archive_changed = False;

    #print('archive changed : '  , archive_changed)  
    

    return archive_changed 


In [13]:
### gauthier


def update_iterations_count(change,iterations_count):
  if change==False:
    iterations_count+=1
    #print(iterations_count)
  else:
    iterations_count=0
    #print(iterations_count)
  return iterations_count


def stop_condition(iterations_count,limite):
  if iterations_count > limite:
    return 0
  else:
    return 1

In [14]:
### here's the main

# nombre de points à l'initialisation
n_init = 500
## nombre d'objectifs
n_obj = 2
## nombre de voisins par voisinage
n_neighbour = 30

## nombre de "chromosomes" permutés d'un voisin
nPermut = 3

# l'archive a été modifiée?
archive_changed = False

# nombre d'itérations
iterations_count=0

#n_max d'itérations sans changements
limite = 10

## lecture de fichier
filepath = 'Data/'
dim, obj = read_data(filepath+'LAP-8-3objSOL.dat')


In [16]:
## initialisation de l'archive
ini = init(dim, n_init)
print(f"""
        dim : {dim} 
        obj : {obj}
        init : {init}
    """)


obj = obj[0:n_obj]


        dim : 8 
        obj : [array([[ 1,  6,  5,  2,  5, 10,  3,  6],
       [ 7,  8,  9,  2,  7,  2,  9,  2],
       [ 7, 10,  3, 10,  3,  6,  3,  2],
       [ 9,  6,  5,  2,  7,  6,  3,  2],
       [ 3,  6,  1,  6,  3, 10,  5,  6],
       [ 7, 10,  5,  4,  5,  2,  3,  6],
       [ 5,  8,  3,  8,  7,  8,  9,  6],
       [ 3,  4,  1,  6,  5,  6,  7,  4]]), array([[ 3, 20, 17,  2, 15,  4, 17,  2],
       [19, 20,  5, 14, 11, 12,  9, 10],
       [19,  4, 17,  2,  3,  8, 13,  6],
       [ 3, 16,  5, 14, 19, 16, 13,  6],
       [19, 20, 17,  6, 19, 16, 17, 10],
       [ 3, 12, 17, 10, 11,  8, 13,  6],
       [19, 16,  1, 18,  3, 20,  1, 10],
       [ 7, 16,  1, 18, 19,  8, 17, 14]]), array([[ 1,  6, 13, 22, 21, 28,  3, 28],
       [ 9, 12, 11, 20, 13, 14,  7,  4],
       [19, 24, 19,  2, 13, 24, 15, 22],
       [ 5,  4,  1, 14, 19, 22,  1,  2],
       [ 1, 30, 21, 26, 13, 30,  9, 18],
       [ 1, 16, 11,  6,  7, 16, 29, 16],
       [ 3, 24, 29,  8, 19, 14, 15, 14],
       [ 5, 26,  9, 

In [17]:
## obtention de tuples + cleanage archive
tupled_ini = x_and_sol_to_named_tuple(ini,obj)
archive = compare_and_delete(tupled_ini)

In [18]:
i = 0 

while (i<1000000) :
    
    #print('iteration #',i)
    
    ## select a random element from the archive
    original_neigh = random.choice(archive)
    #print(original_neigh)

    ## calcul de voisin d'un point de l'archive
    neighbourhood = findNeighboursA(original_neigh.x, n_neighbour,nPermut)

    ## calcul des solutions des voisins
    tupled_neighbourhood = x_and_sol_to_named_tuple(neighbourhood,obj)

    ## ajouter l'original au voisinage
    #tupled_neighbourhood.append(Combo(original_neigh.x,original_neigh.solution))
    tupled_neighbourhood.extend([original_neigh])

    ## Nettoyage du voisinage
    clean_neighbourhood = compare_and_delete(tupled_neighbourhood)

    ## ajouter les voisins à l'archive
    new_archive= add_and_update_archive(clean_neighbourhood,archive)

    ## comparer si l'ancien archive et la nouvelle sont identiques
    archive_changed = compare_old_and_new_archive(archive ,new_archive)
    
    ## nouvelle archive = l'ancienne
    archive = new_archive
    
    if (i%10000 == 0) :
        print('iteration #',i)
        print("LA nvlle archive : ", archive)

    ## condition d'arret
    iterations_count = update_iterations_count(archive_changed, iterations_count)
    test = stop_condition(iterations_count,limite)  
    
    i += 1
    
    if (i == 999999) :
        print('iteration #',i)
        print("LA nvlle archive : ", archive)
    
    if test == False:
        print("test", test , "iterations_count", iterations_count, "limite", limite)
        print("FINAL ARCHIVE : ", archive)
        break


iteration # 0
LA nvlle archive :  [Combo(x=[3, 5, 7, 6, 1, 0, 4, 2], solution=[30, 60]), Combo(x=[3, 5, 4, 1, 7, 0, 6, 2], solution=[36, 48]), Combo(x=[7, 1, 4, 0, 3, 5, 6, 2], solution=[44, 44]), Combo(x=[1, 3, 6, 7, 4, 5, 2, 0], solution=[24, 88]), Combo(x=[1, 3, 4, 7, 0, 5, 6, 2], solution=[28, 72])]
iteration # 10000
LA nvlle archive :  [Combo(x=[0, 3, 4, 7, 1, 5, 6, 2], solution=[26, 56]), Combo(x=[0, 6, 1, 7, 3, 5, 4, 2], solution=[38, 40]), Combo(x=[7, 6, 1, 0, 3, 5, 4, 2], solution=[50, 36]), Combo(x=[0, 3, 6, 7, 4, 5, 2, 1], solution=[20, 80]), Combo(x=[0, 3, 4, 6, 7, 5, 2, 1], solution=[24, 68])]
iteration # 20000
LA nvlle archive :  [Combo(x=[0, 7, 4, 1, 3, 5, 6, 2], solution=[30, 48]), Combo(x=[0, 5, 4, 7, 3, 1, 6, 2], solution=[34, 44]), Combo(x=[6, 3, 1, 7, 4, 5, 2, 0], solution=[28, 76]), Combo(x=[0, 3, 6, 7, 4, 5, 1, 2], solution=[22, 80])]
iteration # 30000
LA nvlle archive :  [Combo(x=[0, 5, 4, 7, 1, 3, 6, 2], solution=[28, 56]), Combo(x=[0, 5, 4, 7, 3, 1, 6, 2], solu

iteration # 330000
LA nvlle archive :  [Combo(x=[0, 6, 4, 7, 3, 5, 2, 1], solution=[30, 52]), Combo(x=[0, 2, 4, 7, 3, 5, 6, 1], solution=[36, 48]), Combo(x=[0, 7, 4, 6, 3, 5, 2, 1], solution=[24, 60]), Combo(x=[5, 2, 4, 7, 3, 0, 6, 1], solution=[50, 44]), Combo(x=[0, 7, 4, 3, 6, 5, 2, 1], solution=[22, 72])]
iteration # 340000
LA nvlle archive :  [Combo(x=[0, 5, 1, 7, 3, 4, 6, 2], solution=[36, 44]), Combo(x=[0, 3, 1, 7, 4, 5, 6, 2], solution=[30, 56]), Combo(x=[5, 4, 1, 7, 3, 0, 6, 2], solution=[52, 36]), Combo(x=[0, 6, 1, 7, 3, 5, 4, 2], solution=[38, 40])]
iteration # 350000
LA nvlle archive :  [Combo(x=[0, 3, 4, 7, 6, 5, 2, 1], solution=[22, 68]), Combo(x=[0, 5, 4, 7, 3, 6, 2, 1], solution=[24, 60]), Combo(x=[0, 3, 4, 7, 1, 5, 6, 2], solution=[26, 56])]
iteration # 360000
LA nvlle archive :  [Combo(x=[0, 7, 4, 1, 3, 5, 6, 2], solution=[30, 48]), Combo(x=[0, 5, 4, 7, 3, 1, 6, 2], solution=[34, 44]), Combo(x=[0, 3, 4, 7, 1, 5, 6, 2], solution=[26, 56]), Combo(x=[0, 3, 4, 7, 6, 5, 2, 

iteration # 660000
LA nvlle archive :  [Combo(x=[0, 3, 4, 7, 2, 5, 6, 1], solution=[24, 68]), Combo(x=[5, 7, 4, 1, 3, 0, 6, 2], solution=[44, 44]), Combo(x=[0, 7, 4, 3, 1, 5, 6, 2], solution=[26, 60]), Combo(x=[0, 7, 4, 1, 3, 5, 6, 2], solution=[30, 48])]
iteration # 670000
LA nvlle archive :  [Combo(x=[0, 3, 4, 7, 6, 5, 2, 1], solution=[22, 68]), Combo(x=[0, 7, 4, 1, 3, 5, 6, 2], solution=[30, 48]), Combo(x=[0, 5, 4, 7, 1, 3, 6, 2], solution=[28, 56]), Combo(x=[0, 5, 4, 7, 3, 1, 6, 2], solution=[34, 44])]
iteration # 680000
LA nvlle archive :  [Combo(x=[0, 3, 4, 7, 6, 5, 2, 1], solution=[22, 68]), Combo(x=[0, 6, 4, 7, 3, 5, 2, 1], solution=[30, 52]), Combo(x=[0, 3, 7, 6, 4, 5, 2, 1], solution=[20, 80]), Combo(x=[0, 7, 4, 6, 3, 5, 1, 2], solution=[26, 60]), Combo(x=[0, 5, 4, 7, 3, 1, 6, 2], solution=[34, 44]), Combo(x=[5, 4, 1, 7, 3, 0, 6, 2], solution=[52, 36]), Combo(x=[0, 6, 1, 7, 3, 5, 4, 2], solution=[38, 40]), Combo(x=[0, 1, 4, 7, 3, 5, 6, 2], solution=[32, 48]), Combo(x=[0, 5, 4

KeyboardInterrupt: 

In [ ]:
archive

### save the file numbers ughuhuh

In [172]:
file = open('nos_solutions\8x83obtry2.txt','w') 

In [173]:
for arch in archive :
    for x in arch.x :
        file.write(str(x))
        file.write(' ')
    file.write('\n')
    print (arch.x)

[0, 4, 3, 1, 7, 5, 6, 2]
[0, 7, 6, 3, 4, 5, 1, 2]
[5, 7, 3, 1, 4, 0, 6, 2]
[0, 7, 6, 1, 4, 5, 3, 2]
[6, 7, 3, 1, 0, 4, 5, 2]
[0, 7, 3, 1, 6, 5, 4, 2]
[0, 7, 3, 6, 1, 5, 4, 2]
[0, 5, 6, 7, 4, 3, 2, 1]
[0, 4, 6, 7, 3, 5, 2, 1]


In [174]:
for arch in archive :
    for xol in arch.solution :
        file.write(str(xol))
        file.write(' ')
    file.write('\n')
    print (arch.solution)

[42, 52, 78]
[22, 84, 96]
[48, 56, 76]
[26, 88, 70]
[38, 96, 44]
[34, 60, 64]
[32, 60, 82]
[22, 80, 106]
[28, 64, 128]


In [ ]:
tupled_ini = x_and_sol_to_named_tuple(ini,obj)

In [ ]:
type(tupled_ini)

In [ ]:
tupled_ini

In [ ]:
archive = compare_and_delete(tupled_ini)

In [ ]:
type(archive)

In [ ]:
archive

In [ ]:
#### JUSQU ICI OK

In [ ]:
original_neigh = random.choice(archive)
print(original_neigh)

In [ ]:
type(original_neigh)

In [ ]:
####Ok

In [ ]:
## calcul de voisin d'un point de l'archive
neighbourhood = findNeighbours(original_neigh.x, n_neighbour)


In [ ]:
type(original_neigh.x)

In [ ]:
original_neigh.x

In [ ]:
type(neighbourhood)

In [ ]:
neighbourhood

In [ ]:
#### jusqu'ici ok.

In [ ]:
## calcul des solutions des voisins
tupled_neighbourhood = x_and_sol_to_named_tuple(neighbourhood,obj)


In [ ]:
print(obj)
print(type(obj))

In [ ]:
print(tupled_neighbourhood)

In [ ]:
print(type(tupled_neighbourhood))
print(type(tupled_neighbourhood[0]))

In [ ]:
#### jusqu'ici ok

In [ ]:
## ajouter l'original au voisinage
    #tupled_neighbourhood.append(Combo(original_neigh.x,original_neigh.solution))
tupled_neighbourhood.extend([original_neigh])

In [ ]:
print(tupled_neighbourhood)

In [ ]:
print(type(tupled_neighbourhood))
print(type(tupled_neighbourhood[0]))

In [ ]:
#### OK

In [ ]:
## Nettoyage du voisinage
clean_neighbourhood = compare_and_delete(tupled_neighbourhood)


In [ ]:
print(clean_neighbourhood)

In [ ]:
print(type(clean_neighbourhood))

In [ ]:
##" okok

In [ ]:
## ajouter les voisins à l'archive
new_archive= add_and_update_archive(clean_neighbourhood,archive)

In [ ]:
print(new_archive)

In [ ]:
###NE DEVRAIT PAS ETRE TUPLE LOL.
print(type(new_archive))

In [ ]:
## comparer si l'ancien archive et la nouvelle sont identiques
archive_changed = compare_old_and_new_archive(archive ,new_archive)
    


In [ ]:
print(archive_changed)

In [ ]:
## nouvelle archive = l'ancienne
archive = new_archive

In [ ]:
print(archive)

In [28]:
### how to compare 2 archives ?

x1 = [0,1,2,3,4,5,7,6]
x2 = [0,1,2,3,4,5,6,7]
x2 = [7,6,5,4,3,2,1,0]
x3 = [0,1,4,6,5,3,2,7]
x4 = [0,1,4,7,6,5,3,2]
x5 = [0,1,2,3,6,7,4,5]

#[x1,x2,x3,x4,x5] = x_and_sol_to_named_tuple([x1,x2,x3,x4,x5],obj)
      
x1 = x_and_sol_to_named_tuple(x1,obj)
x2 = x_and_sol_to_named_tuple(x2,obj)
x3 = x_and_sol_to_named_tuple(x3,obj)
x4 = x_and_sol_to_named_tuple(x4,obj)
x5 = x_and_sol_to_named_tuple(x5,obj)     

In [29]:
x1

[Combo(x=[0, 1, 2, 3, 4, 5, 7, 6], solution=[32, 108, 108])]

In [30]:
x2

[Combo(x=[7, 6, 5, 4, 3, 2, 1, 0], solution=[50, 84, 144])]

In [31]:
x3

[Combo(x=[0, 1, 4, 6, 5, 3, 2, 7], solution=[36, 80, 102])]

In [32]:
x4

[Combo(x=[0, 1, 4, 7, 6, 5, 3, 2], solution=[30, 76, 70])]

In [33]:
x5

[Combo(x=[0, 1, 2, 3, 6, 7, 4, 5], solution=[38, 88, 108])]

In [48]:
perfect = Combo(x=[0,1,2,3,4,5,6], solution =[0,0,0])

In [49]:
worst = Combo(x=[0,1,2,3,4,5,6], solution =[1000,1000,1000])

In [159]:
hup1 = Combo(x=[0,1,2,3,4,5,6], solution =[42,84,58])
hup2= Combo(x=[0,1,2,3,4,5,6], solution =[34,88,62])
hup3 = Combo(x=[0,1,2,3,4,5,6], solution =[30,96,54])
hup4 = Combo(x=[0,1,2,3,4,5,6], solution =[34,56,90])
hup5 = Combo(x=[0,1,2,3,4,5,6], solution =[28,104,72])
hup6 = Combo(x=[0,1,2,3,4,5,6], solution =[34,60,64])

hupa = Combo(x=[0,1,2,3,4,5,6], solution =[28,68,84])
hupb = Combo(x=[0,1,2,3,4,5,6], solution =[28,100,78])
hupc = Combo(x=[0,1,2,3,4,5,6], solution =[52,44,116])
hupd = Combo(x=[0,1,2,3,4,5,6], solution =[52,64,68])

In [160]:
hupletter =  [hupa,hupb,hupc,hupd]
hupdecib =[hup1,hup2,hup3,hup4,hup5,hup6]

In [164]:
add_and_update_archive(hupletter,hupdecib)

[Combo(x=[0, 1, 2, 3, 4, 5, 6], solution=[42, 84, 58]),
 Combo(x=[0, 1, 2, 3, 4, 5, 6], solution=[34, 88, 62]),
 Combo(x=[0, 1, 2, 3, 4, 5, 6], solution=[30, 96, 54]),
 Combo(x=[0, 1, 2, 3, 4, 5, 6], solution=[34, 56, 90]),
 Combo(x=[0, 1, 2, 3, 4, 5, 6], solution=[28, 104, 72]),
 Combo(x=[0, 1, 2, 3, 4, 5, 6], solution=[34, 60, 64]),
 Combo(x=[0, 1, 2, 3, 4, 5, 6], solution=[28, 68, 84]),
 Combo(x=[0, 1, 2, 3, 4, 5, 6], solution=[28, 100, 78]),
 Combo(x=[0, 1, 2, 3, 4, 5, 6], solution=[52, 44, 116])]

In [ ]:
print(archive_old)

In [ ]:
equivalent_found = 0
archive_changed = True;

for new in archive_new :   
    
    for old in archive_old :    
        
        #print(old[0].x)
        #print(type(old))
        #print(new.x)
        
        if old[0].x == new[0].x :
            
            equivalent_found += 1
            continue;
print("number of equivalences found : ", equivalent_found)
if (equivalent_found == len(archive_new) & len(archive_old) == len(archive_new)):
    archive_changed = False;
            
print('archive changed : '  , archive_changed)   

return archive_changed

In [ ]:
set_1 = set(archive_1)
set_2 = set(archive_2)

In [ ]:
archive_1 == archive_2

In [ ]:
#newNeighbour[permut_rows[0]] = newNeighbour[permut_rows[-1]]
#print(newNeighbour)

In [ ]:
for tmp,row in zip(reversed(temps),permut_rows) :   
    newNeighbour[row] = tmp
    print(newNeighbour)
    

#tmp = newNeighbour[row]
    
         
    

In [ ]:
newNeighbour = [0,1,2,3,4,5,6,7]
nbNeighbour = 5

In [ ]:
findNeighbours(newNeighbour, nbNeighbour,5)

In [ ]:
[row1,row2,row3,row4] = random.sample(X,4)

In [ ]:
x1

In [ ]:
x2

In [ ]:
x3

In [ ]:
x4

In [ ]:
x5

In [43]:
compare_two_points(x2[0],x5[0])

0

In [ ]:
all(x in {0, 1, -1} for x in [0,1])

In [ ]:
all(x in [0,1] for x in {0, 1, -1})

In [6]:
i = 0
while True :
    i += 1
    
    if (i%100 == 0) :
        print("coucou :" , i)
        
    if i == 100000 :
        break;


coucou : 100
coucou : 200
coucou : 300
coucou : 400
coucou : 500
coucou : 600
coucou : 700
coucou : 800
coucou : 900
coucou : 1000
coucou : 1100
coucou : 1200
coucou : 1300
coucou : 1400
coucou : 1500
coucou : 1600
coucou : 1700
coucou : 1800
coucou : 1900
coucou : 2000
coucou : 2100
coucou : 2200
coucou : 2300
coucou : 2400
coucou : 2500
coucou : 2600
coucou : 2700
coucou : 2800
coucou : 2900
coucou : 3000
coucou : 3100
coucou : 3200
coucou : 3300
coucou : 3400
coucou : 3500
coucou : 3600
coucou : 3700
coucou : 3800
coucou : 3900
coucou : 4000
coucou : 4100
coucou : 4200
coucou : 4300
coucou : 4400
coucou : 4500
coucou : 4600
coucou : 4700
coucou : 4800
coucou : 4900
coucou : 5000
coucou : 5100
coucou : 5200
coucou : 5300
coucou : 5400
coucou : 5500
coucou : 5600
coucou : 5700
coucou : 5800
coucou : 5900
coucou : 6000
coucou : 6100
coucou : 6200
coucou : 6300
coucou : 6400
coucou : 6500
coucou : 6600
coucou : 6700
coucou : 6800
coucou : 6900
coucou : 7000
coucou : 7100
coucou : 7200
c